In [105]:
from __future__ import print_function, division
import os
import torch
import pandas as pd              #用于更容易地进行csv解析
import skimage
from skimage import io, transform    #用于图像的IO和变换
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

# 忽略警告
import warnings
warnings.filterwarnings("ignore")

plt.ion()   # interactive mode

In [95]:
absolute_path = os.path.abspath('./')
absolute_path

'/Users/derenzhu/Github/my_github/Python_Module/Three_Part_Moudule/pytorch'

In [101]:
landmarks_frame = pd.read_csv(absolute_path +'/faces/face_landmarks.csv')
# face_landmarks.csv contain img_name and all point positions at face
n = 0
img_name = landmarks_frame.iloc[n, 0]
landmarks = landmarks_frame.iloc[n, 1:].values
landmarks = landmarks.astype('float').reshape(-1, 2) # 转换称x轴和y轴坐标的形式

print('Image name: {}'.format(img_name))
print('Landmarks shape: {}'.format(landmarks.shape))
print('First 4 Landmarks: {}'.format(landmarks[:4]))

def show_landmarks(image, landmarks):
    plt.imshow(image)
    # 散点图，x=x轴, y=y轴 s=散点图中点的大小，c=颜色，marker=类型，alpha=透明度
    plt.scatter(landmarks[:, 0], landmarks[:, 1], s=10, marker='+', c='r')
    plt.pause(0.001)

plt.figure()
show_landmarks(
    io.imread(os.path.join(absolute_path + '/faces/', img_name)),
    landmarks)

plt.show()

FileNotFoundError: [Errno 2] File b'/Users/derenzhu/Github/my_github/Python_Module/Three_Part_Moudule/pytorch/faces/face_landmarks.csv' does not exist: b'/Users/derenzhu/Github/my_github/Python_Module/Three_Part_Moudule/pytorch/faces/face_landmarks.csv'

In [102]:
# 建立数据集类
class FaceLandMarksDataset(Dataset):
    
    def __init__(self, csv_file, root_dir, transform=None):
        self.landmarks_frame = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform
    
    def __len__(self):
        return len(self.landmarks_frame)
    
    def __getitem__(self, idx):
        img_name = os.path.join(
            self.root_dir,
            self.landmarks_frame.iloc[idx, 0]
        )
        image = io.imread(img_name)
        landmarks = self.landmarks_frame.iloc[idx, 1:]
        landmarks = np.array([landmarks])
        landmarks = landmarks.astype('float').reshape(-1, 2)
        sample = {'image': image, 'landmarks': landmarks}
        
        if self.transform:
            sample = self.transform(sample)
        return sample

In [103]:
csv_file = os.path.abspath('./') + '/faces/face_landmarks.csv'
root_dir = 'faces/'
face_dataset = FaceLandMarksDataset(csv_file, root_dir)

fig = plt.figure()
for i in range(len(face_dataset)):
    sample = face_dataset[i]
    
    print(i, sample['image'].shape, sample['landmarks'].shape)
    
    ax = plt.subplot(1, 4, i + 1)
    plt.tight_layout()
    ax.set_title('Sample #{}'.format(i))
    ax.axis('off')
    show_landmarks(**sample)
    
    if i == 3:d
        plt.show()
        break

IndentationError: unexpected indent (<ipython-input-103-131ea7146416>, line 18)

In [48]:
# 数据变换 Transform

In [66]:
class Rescale(object):
    """ 将样本中的图像重新缩放到给定大小 """
    def __init__(self, output_size):
        assert isinstance(output_size, (int, tuple))
        self.output_size = output_size
    
    def __call__(self, sample):
        image, landmarks = sample['image'], sample['landmarks']
        
        h, w = image.shape[:2]
        new_h, new_w = h, w
        if isinstance(self.output_size, int):
            if h > w:
                new_h, new_h = self.output_size * h / w, self.output_size
            else:
                new_h, new_w = self.output_size, self.output_size * w / h
        else:
            new_h, new_w = self.output_size
        new_h, new_w = int(new_h), int(new_w)
        img = skimage.transform.resize(image, (new_h, new_w))
        landmarks = landmarks * [new_w / w, new_h / h]
        
        return {'image': img, 'landmarks': landmarks}

    
class RandomCrop(object):
    """ 随机裁剪样本中的图像 """
    def __init__(self, output_size):
        assert isinstance(output_size, (int, tuple))
        if isinstance(output_size, int):
            self.output_size = (output_size, output_size)
        else:
            assert len(output_size) == 2
            self.output_size = output_size
    
    def __call__(self, sample):
        image, landmarks = sample['image'], sample['landmarks']
        h, w = image.shape[:2]
        new_h, new_w = self.output_size
        top = np.random.randint(0, h - new_h)
        left = np.random.randint(0, w - new_w)
        image = image[
            top: top + new_h, 
            left: left + new_w]
        landmarks = landmarks - [left, top]
        return {'image': image, 'landmarks': landmarks}

class ToTensor(object):
    """ 将样本中的ndarray转换为Tensors. """
    def __call__(self, sample):
        image, landmarks = sample['image'], sample['landmarks']
        image = image.transform((2, 0, 1))
        return {
            'image': torch.from_numpy(image),
            'landmarks': torch.from_numpy(landmarks)
        }

In [104]:
# 组合转换
scale = Rescale(256)
crop = RandomCrop(128)
composed = transforms.Compose([
    Rescale(256),
    RandomCrop(224)
])

fig = plt.figure()
sample = face_dataset[0]
print(sample['image'].shape[0])
for i, transform in enumerate([scale, crop, composed]):
    transformed_sample = transform(sample)
    
    ax = plt.subplot(1, 3, i + 1)
    plt.tight_layout()
    ax.set_title(type(transform).__name__)
    show_landmarks(**transformed_sample)

plt.show()

FileNotFoundError: [Errno 2] No such file or directory: 'faces/0805personali01.jpg'

<Figure size 432x288 with 0 Axes>